In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, transforms
from torchvision.models import resnet50, ResNet50_Weights
import matplotlib.pyplot as plt
import time
import os
import copy

In [ ]:
transform_training_set = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
image_dir = 'sex_normal_toy'

In [ ]:
training_set = datasets.ImageFolder(os.path.join(image_dir, 'training'), transform_training_set)

In [ ]:
training_data = torch.utils.data.DataLoader(training_set, shuffle=True, num_workers=4)

In [ ]:
len(training_data)

In [ ]:
image_labels = training_set.classes

In [ ]:
image_labels

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
resnet_model = resnet50(weights=ResNet50_Weights.DEFAULT)
number_features = resnet_model.fc.in_features
resnet_model.fc = nn.Linear(number_features, len(image_labels))
resnet_model = resnet_model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet_model.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

since = time.time()
for epoch in range(10):
    resnet_model.train()
    print(f'Epoch {epoch}/{5 - 1}')
    print('-' * 10)
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in training_data:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        # forward
        # track history if only in train
#         torch.set_grad_enabled(True)
        outputs = resnet_model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # backward + optimize only if in training phase
        loss.backward()
        optimizer.step()

        # statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        
    scheduler.step()
    epoch_loss = running_loss / len(training_data)
    epoch_acc = running_corrects.double() / len(training_data)

    print(f'Training Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
    
time_elapsed = time.time() - since
print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')

In [ ]:
from PIL import Image

In [ ]:
transform_validation_set = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
resnet_model.eval()
validation_image = Image.open('sex_toy.jpeg')
test_image_Data = transform_validation_set(validation_image).unsqueeze(0).to(device)
out = resnet_model(x)
p = torch.nn.functional.softmax(out, dim=1)
score, index = torch.topk(p, 1)
input_category_id = index[0][0].item() # 716
predicted_confidence = score[0][0].item()
print(f'label: {image_labels[input_category_id]} confidence: {predicted_confidence}')

In [ ]:
validation_image = Image.open('normal_toy.jpeg')
test_image_Data = transform_validation_set(validation_image).unsqueeze(0).to(device)
out = resnet_model(x)
p = torch.nn.functional.softmax(out, dim=1)
score, index = torch.topk(p, 1)
input_category_id = index[0][0].item() # 716
predicted_confidence = score[0][0].item()
print(f'label: {image_labels[input_category_id]} confidence: {predicted_confidence}')